In [1]:
import pandas as pd
import seaborn as sns 
from pandas import DataFrame
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder
import numpy as np
import plotly
import statistics
import plotly.express as px
import stats
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import plotly.express as px
import datetime
import datetime as dt
import time


# Supress Scientific notation in python
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Display all columns of long dataframe
pd.set_option('display.max_columns', None)

#data from Kaggle.com
#open file
stocks='stocks.csv'
df=pd.read_csv(stocks)
print(df)

                             Stock Name   Code       Date  Price  Open  High  \
0       Abra Mining and Industrial Corp     AR   3-Mar-21      0     0     0   
1       Abra Mining and Industrial Corp     AR   2-Mar-21      0     0     0   
2       Abra Mining and Industrial Corp     AR   1-Mar-21      0     0     0   
3       Abra Mining and Industrial Corp     AR  26-Feb-21      0     0     0   
4       Abra Mining and Industrial Corp     AR  24-Feb-21      0     0     0   
...                                 ...    ...        ...    ...   ...   ...   
481915                Ayala Corp Pref B  ACPB2  14-Nov-14    503   504   504   
481916                Ayala Corp Pref B  ACPB2  12-Nov-14    508   508   508   
481917                Ayala Corp Pref B  ACPB2  11-Nov-14    514   509   515   
481918                Ayala Corp Pref B  ACPB2  10-Nov-14    509   509   509   
481919                Ayala Corp Pref B  ACPB2   7-Nov-14    511   512   512   

        Low   Volume  Change%  
0      

In [2]:
#parse index
df['Date']=pd.to_datetime(df['Date'], infer_datetime_format=True)
indexeddf=df.set_index(['Date'])
#print(indexeddf)

#parsing to time format and extracting dates 
x=df['Date']=pd.to_datetime(df['Date'], format='%d-%m-%y')

Day=df['Date'].dt.day_name()
Month=df['Date'].dt.month_name()
Year=df['Date'].dt.year

df['Year']=df['Date'].dt.year
df['Month']=df['Date'].dt.month_name()
df['Day']=df['Date'].dt.day_name()
print(df)

                             Stock Name   Code       Date  Price  Open  High  \
0       Abra Mining and Industrial Corp     AR 2021-03-03      0     0     0   
1       Abra Mining and Industrial Corp     AR 2021-03-02      0     0     0   
2       Abra Mining and Industrial Corp     AR 2021-03-01      0     0     0   
3       Abra Mining and Industrial Corp     AR 2021-02-26      0     0     0   
4       Abra Mining and Industrial Corp     AR 2021-02-24      0     0     0   
...                                 ...    ...        ...    ...   ...   ...   
481915                Ayala Corp Pref B  ACPB2 2014-11-14    503   504   504   
481916                Ayala Corp Pref B  ACPB2 2014-11-12    508   508   508   
481917                Ayala Corp Pref B  ACPB2 2014-11-11    514   509   515   
481918                Ayala Corp Pref B  ACPB2 2014-11-10    509   509   509   
481919                Ayala Corp Pref B  ACPB2 2014-11-07    511   512   512   

        Low   Volume  Change%  Year    

In [7]:
# Read the file into a variable flight_data
stock_data = pd.read_csv(stocks, index_col="Date")

In [8]:
stock_data

,Stock Name,Code,Price,Open,High,Low,Volume,Change%
Date,,,,,,,,
3-Mar-21,Abra Mining and Industrial Corp,AR,0,0,0,0,215.45B,-14.81%
2-Mar-21,Abra Mining and Industrial Corp,AR,0,0,0,0,122.74B,50.00%
1-Mar-21,Abra Mining and Industrial Corp,AR,0,0,0,0,5.89B,5.88%
26-Feb-21,Abra Mining and Industrial Corp,AR,0,0,0,0,9.59B,0.00%
24-Feb-21,Abra Mining and Industrial Corp,AR,0,0,0,0,29.68B,-12.82%
...,...,...,...,...,...,...,...,...
14-Nov-14,Ayala Corp Pref B,ACPB2,503,504,504,503,0.57K,-0.98%
12-Nov-14,Ayala Corp Pref B,ACPB2,508,508,508,508,0.10K,-1.17%
11-Nov-14,Ayala Corp Pref B,ACPB2,514,509,515,507,5.50K,0.98%
